In [ ]:
from astropy import units as u
from astropy.coordinates import EarthLocation
from astropy.time import Time
from astropy.table import Table, join
from astropy.coordinates import SkyCoord
from astropy.io import ascii

from astroplan import Observer, FixedTarget
from astroplan import (AltitudeConstraint, AirmassConstraint, AtNightConstraint)
from astroplan import is_observable, is_always_observable, months_observable
from astroplan import observability_table
from astroplan import time_grid_from_range
from astroplan.plots import plot_sky, plot_airmass, plot_altitude
import astroplan

from collections import Counter
from math import radians, cos, log10
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patheffects as path_effects
from matplotlib import cm
import pandas as pd
import warnings 

main_catalogue_matched = Table.read('../data/catalogos/main_catalogue_matched_GAIA.csv', format='csv')

main_catalogue_no_vr = main_catalogue_matched[main_catalogue_matched['radial_velocity'].mask == True]
main_catalogue_no_vr.write('./tables/match_gaia_no_vr.csv', overwrite=True)
print("Estrellas del match con GAIA sin VR:", len(main_catalogue_no_vr))

catalogue = Table.read('./tables/all_stars_without_vr.csv', format='csv')
print("Total de estrellas sin VR en ninguno de los sondeos:", len(catalogue))


print("Los observatorios disponibles son: CASLEO, SPM, KPNO y SOAR")
location = input("Desde donde desea observar?")

if location == "CASLEO":
    location = Observer(longitude=-69.2956*u.deg, latitude=-31.798*u.deg, elevation=2552*u.m, name="CASLEO")
    min_mag = 9
    max_mag = 13
elif location == "SPM":
    location = Observer.at_site("spm")
    min_mag = 0
    max_mag = 12
elif location == "KPNO":
    location = Observer.at_site("Kitt Peak")
    min_mag = 0
    max_mag = 15
elif location == "SOAR":
    location = Observer.at_site("Cerro Pachon")
    min_mag = 14.9
    max_mag = 17.7

time_range = Time(["2021-01-01 00:00", "2021-12-31 23:59"]) # Inicializamos para observar 24/7 durante todo un año

# ---------------------- --------------------- #
#  DEFINIMOS LAS RESTRICCIONES OBSERVACIONALES #
# -------------------------------------------- #

# Calculamos la masa de aire
az_degrees = 45
az_radians = radians(az_degrees)
airmass = 1/(cos(az_radians))

print("Masa de aire:", airmass)

constraints = [AirmassConstraint(airmass), AtNightConstraint.twilight_civil()]

# --------------------- #
#  FILTRADOS Y TARGETS  #
# --------------------- #  

targets_no_vr = [FixedTarget(coord=SkyCoord(ra=RA*u.deg, dec=DEC*u.deg), name=ID, ra=RA, dec=DEC)
               for ID,RA,DEC,YNMG,phot_g_mean_mag,phot_bp_mean_mag,phot_rp_mean_mag,radial_velocity,teff_val,APOGEE_ID,VHELIO_AVG,
                   galah_sobject_id,rv_galah,lamost_mobsid,rv_b0,rv_r0
                in catalogue]

warnings.filterwarnings('ignore')

In [ ]:
obs_table = astroplan.observability_table(constraints, location, targets_no_vr, time_range=time_range)
obs_table.rename_column('target name', 'ID')
obs_table.rename_column('ever observable', 'ever_obs')
obs_table.rename_column('always observable', 'always_obs')
obs_table.rename_column('fraction of time observable', 'fraction_obs')
catalogue_obs = join(catalogue, obs_table)

catalogue_ever_obs = catalogue_obs[(catalogue_obs['ever_obs'] == True)]

In [ ]:
main_catalogue = catalogue_ever_obs[(catalogue_ever_obs['phot_g_mean_mag'] < max_mag) & (catalogue_ever_obs['phot_g_mean_mag'] > min_mag)]
print("Estrellas que pueden ser observadas en algún momento:", len(main_catalogue))

In [ ]:
# -------------------------------------------------- #
#    ANÁLISIS DE TODAS LAS ESTRELLAS OBSERVABLES     #
# -------------------------------------------------- #

def group_num_dict(catalogue):
    groups = {}
    for star in catalogue:
        key = star['YNMG']
        if key not in groups:
            groups[key] = 0
        groups[key] += 1
    
    return groups

total_groups = group_num_dict(catalogue)

# Agrupaciones por cantidad en forma de diccionario
total_obs_num_groups = group_num_dict(main_catalogue)
total_obs_num_groups_sorted = sorted(total_obs_num_groups.items(), key=lambda item: item[1], reverse=True)

# Agrupaciones por nombre en forma de diccionario

def group_name_dict(catalogue):
    total_obs_name_groups = {}
    for star in catalogue:
        key = star['YNMG']
        value = star['ID']
        if key not in total_obs_name_groups:
            total_obs_name_groups[key] = []
        total_obs_name_groups[key].append(value)
    
    return total_obs_name_groups

total_obs_name_groups = group_name_dict(main_catalogue)

# ---------------------------- #
#    CREACIÓN DE TABLA FINAL   #
# ---------------------------- #

nymg = [] # array con los nymg
total_amount = [] # array con el total de los nymg
           
def build_amount(array, dic): # devuelve un array con dic.values de dic.keys que pertenecen al array
    amount = []           
    for group in array:
        if group in dic.keys():
            amount.append(dic[group])
        else:
            amount.append(0)
    return amount

# Categorizamos por grupos inciertos

uncertain_amount = 0 # cantidad total de estrellas con agrupaciones inciertas
uncertain_groups = {} # agrupaciones inciertas y sus cantidades
for key, value in total_groups.items():
    if ';' in key or '_' in key or '-' in key or 'Ambiguous' in key:
        uncertain_amount += total_groups[key]
        uncertain_groups[key] = total_groups[key]
        next
    else:
        nymg.append(key)
        total_amount.append(value)

total_no_vr_groups = group_num_dict(catalogue) # dicccionario con los nymg sin vr y sus cantidades 

total_no_vr_amount = build_amount(nymg, total_no_vr_groups) # array con el total de candidatos sin vr    

def uncertain(dic1, dic2): # cantidad de elementos en dic1 que hay en dic2
    amount = 0
    for uncertain in dic1.keys():
        if uncertain in dic2.keys():
            amount += dic2[uncertain]
    return amount

uncertain_no_vr_amount = uncertain(uncertain_groups,total_no_vr_groups) # vemos cuantos no tienen vr

# Agregamos la info asociada a los uncertains a cada columna
total_amount.append(uncertain_amount)
total_no_vr_amount.append(uncertain_no_vr_amount)

if location.name == 'spm':
    observable_spm = build_amount(nymg, total_obs_num_groups)     
    observable_spm.append(uncertain(uncertain_groups, total_obs_num_groups))
if location.name == 'CASLEO':
    observable_casleo = build_amount(nymg, total_obs_num_groups)
    observable_casleo.append(uncertain(uncertain_groups, total_obs_num_groups))       
if location.name == 'Kitt Peak':
    observable_kpno = build_amount(nymg, total_obs_num_groups)
    observable_kpno.append(uncertain(uncertain_groups, total_obs_num_groups))       
if location.name == 'Cerro Pachon':
    observable_soar = build_amount(nymg, total_obs_num_groups)   
    observable_soar.append(uncertain(uncertain_groups, total_obs_num_groups))

nymg.append('Uncertain')
    
if observable_spm != [] and observable_casleo != [] and observable_kpno != [] and observable_soar != []:
    resume_table = Table([nymg, total_amount, total_no_vr_amount, observable_spm, observable_casleo, observable_kpno, observable_soar],
                      names=('YNMG', 'total_amount', 'total_no_vr_amount', 'observable_spm', 'observable_casleo', 'observable_kpno', 'observable_soar'))
    #resume_table.add_row(['Uncertain', uncertain_amount, uncertain_li_amount, uncertain_spm, uncertain_casleo, uncertain_kpno, uncertain_soar])
    resume_table.add_row(['TOTAL', np.sum(total_amount), np.sum(total_no_vr_amount), np.sum(observable_spm), np.sum(observable_casleo),
                          np.sum(observable_kpno), np.sum(observable_soar)])
    ascii.write(resume_table, format='latex')  
    resume_table.write('./tables/resume_table_no_vr.csv', overwrite=True)
    resume_table.write('./tables/resume_table_no_vr.fits', overwrite=True)

In [ ]:
# --------------------------------------- #
#   CALCULAMOS EL TIEMPO DE INTEGRACIÓN   #
# --------------------------------------- #

exposure_times = Table.read('../data/exposure_times/exposure_times_obs.csv', format='csv')   

# Filtramos por información del observatorio elegido
if location.name == 'CASLEO':
    exposure_times_location = exposure_times[exposure_times['observatory'] == 'casleo']
elif location.name == 'Cerro Pachon':
    exposure_times_location = exposure_times[exposure_times['observatory'] == 'soar']
elif location.name == 'Kitt Peak':
    exposure_times_location = exposure_times[exposure_times['observatory'] == 'kpno']
else:
    exposure_times_location = exposure_times[exposure_times['observatory'] == location.name]

exposure_times_sec = np.round(np.interp(main_catalogue['phot_g_mean_mag'], exposure_times_location['mag_V'], exposure_times_location['exposure_time_sec']), 0)
exposure_times_sec = exposure_times_sec.astype(int)
total_exposure_time_sec = int(np.sum(exposure_times_sec))
            
total_exposure_time_hrs = np.round(total_exposure_time_sec / 3600, 1);
            
print(f"El tiempo total de integración necesario para observar las {len(main_catalogue)} estrellas es:", total_exposure_time_sec, "segundos.")
print("Su equivalente en horas es:", total_exposure_time_hrs)

#total_main_catalogue.remove_column('exposure_time_sec')
if 'exposure_time_sec' not in main_catalogue.columns:
    main_catalogue['exposure_time_sec'] = exposure_times_sec

# Tiempos de integración por agrupación observable
total_obs_exposure_times_groups = {}
for key in total_obs_num_groups.keys():
    exposure_time_sec_per_key = 0
    for target in main_catalogue:
        if key == target['YNMG']:             
            exposure_time_sec_per_key += target['exposure_time_sec']
    total_obs_exposure_times_groups[key] = exposure_time_sec_per_key

print("Tiempos de integración para observar la totalidad de las poblaciones observables")
uncertain_exposure_times = 0
for key, value in total_obs_exposure_times_groups.items():        
    if key in uncertain_groups.keys():
        uncertain_exposure_times += value
    else:
        print(key, ":", value, "seg /", np.round(value / 3600, 1), "hrs")
        
if uncertain_exposure_times != 0:        
    print("Uncertain:", uncertain_exposure_times, "seg /", np.round(uncertain_exposure_times / 3600, 1), "hrs")
    
# ------------------------------------------------------ #
#   CONSTURIMOS UNA TABLA CON LOS TIEMPOS DE EXPOSICION  #
# ------------------------------------------------------ #

groups_obs = [] #array con las agrupaciones observables
groups_number_obs = [] #array con la cantidad de estrellas en agrupaciones observables
for key,value in total_obs_num_groups.items():
    if key not in uncertain_groups.keys():
        groups_obs.append(key)
        groups_number_obs.append(value)  

exposure_time_per_group_sec = []
for key, value in total_obs_exposure_times_groups.items():
    if key not in uncertain_groups.keys():
        exposure_time_per_group_sec.append(value)
    
uncertain_groups_num = uncertain(uncertain_groups, total_obs_num_groups);
if uncertain_groups_num != 0:
    groups_obs.append('Uncertain')
    groups_number_obs.append(uncertain_groups_num)
    exposure_time_per_group_sec.append(uncertain_exposure_times)
    
exposure_time_per_group_hrs = [np.round(i / 3600, 1) for i in exposure_time_per_group_sec]

if location.name == 'spm':
    overhead_spm = 600 #10 min
    total_overhead = [np.round((i * overhead_spm) / 3600, 1) for i in groups_number_obs]
    total_obs_time_per_group_hrs = np.round(np.add(exposure_time_per_group_hrs, total_overhead), 1)    
    exposure_times_table_spm = Table([groups_obs, groups_number_obs, exposure_time_per_group_hrs, total_obs_time_per_group_hrs],
                                      names=('YNMG', 'total_amount', 'exposure_time_hrs', 'total_time_hrs'))
    exposure_times_table_spm.add_row(['TOTAL', np.sum(groups_number_obs), total_exposure_time_hrs, np.round(np.sum(total_obs_time_per_group_hrs), 2)])
    ascii.write(exposure_times_table_spm, format='latex')  
    exposure_times_table_spm.write('./tables/exposure_times_table_spm_no_vr.csv', overwrite=True)
    
if location.name == 'CASLEO':
    exposure_times_table_casleo = Table([groups_obs, groups_number_obs, exposure_time_per_group_hrs],
                                      names=('YNMG', 'total_amount', 'exposure_time_hrs'))
    exposure_times_table_casleo.add_row(['TOTAL', np.sum(groups_number_obs), total_exposure_time_hrs])
    ascii.write(exposure_times_table_casleo, format='latex')  
    exposure_times_table_casleo.write('./tables/exposure_times_table_casleo_no_vr.csv', overwrite=True)
    
if location.name == 'Kitt Peak':
    overhead_kpno = 300 #5 min
    total_overhead = [np.round((i * overhead_kpno) / 3600, 1) for i in groups_number_obs]
    total_obs_time_per_group_hrs = np.round(np.add(exposure_time_per_group_hrs, total_overhead), 1)    
    exposure_times_table_kpno = Table([groups_obs, groups_number_obs, exposure_time_per_group_hrs, total_obs_time_per_group_hrs],
                                      names=('YNMG', 'total_amount', 'exposure_time_hrs', 'total_time_hrs'))
    exposure_times_table_kpno.add_row(['TOTAL', np.sum(groups_number_obs), total_exposure_time_hrs, np.round(np.sum(total_obs_time_per_group_hrs), 2)])
    ascii.write(exposure_times_table_kpno, format='latex')  
    exposure_times_table_kpno.write('./tables/exposure_times_table_kpno_no_vr.csv', overwrite=True)
    
if location.name == 'Cerro Pachon':
    overhead_soar = 144 #2.4 min
    total_overhead = [np.round((i * overhead_soar) / 3600, 1) for i in groups_number_obs]
    total_obs_time_per_group_hrs = np.round(np.add(exposure_time_per_group_hrs, total_overhead), 1)
    exposure_times_table_soar = Table([groups_obs, groups_number_obs, exposure_time_per_group_hrs, total_obs_time_per_group_hrs],
                                      names=('YNMG', 'total_amount', 'exposure_time_hrs', 'total_time_hrs'))
    exposure_times_table_soar.add_row(['TOTAL', np.sum(groups_number_obs), total_exposure_time_hrs, np.round(np.sum(total_obs_time_per_group_hrs), 2)])
    ascii.write(exposure_times_table_soar, format='latex')  
    exposure_times_table_soar.write('./tables/exposure_times_table_soar_no_vr.csv', overwrite=True)  

In [ ]:
# ---------------------------------------------------------------- #
#   CONSTURIMOS UNA TABLA CON LOS CANDIDATOS DE BPMG Y UNCERTAIN   #
# ---------------------------------------------------------------- #

def bpmg_from_obs():
    bpmg_table = main_catalogue[main_catalogue['YNMG'] == 'BPMG']
    bpmg_table.keep_columns(['ID', 'RA', 'DEC', 'phot_g_mean_mag', 'radial_velocity', 'teff_val', 'radius_val', 'lum_val', 'distances', 'fraction_obs', 'exposure_time_sec'])
    bpmg_table['RA'] = np.round(bpmg_table['RA'], 4)
    bpmg_table['DEC'] = np.round(bpmg_table['DEC'], 3)
    bpmg_table['phot_g_mean_mag'] = np.round(bpmg_table['phot_g_mean_mag'], 2)
    bpmg_table['teff_val'] = np.round(bpmg_table['teff_val'], 0)
    bpmg_table['distances'] = np.round(bpmg_table['distances'], 2)
    bpmg_table['fraction_obs'] = np.round(bpmg_table['fraction_obs'] * 100, 1)
    
    return bpmg_table

def uncertain_from_obs():
    uncertain_id = []
    uncertain_nymg = []
    uncertain_ra = []
    uncertain_dec = []
    uncertain_mag = []
    uncertain_rv = []
    uncertain_teff = []
    uncertain_rad = []
    uncertain_lum = []
    uncertain_dist = []
    uncertain_obs = []
    uncertain_exp = []

    for target in main_catalogue:
        if target['YNMG'] in uncertain_groups.keys():
            uncertain_id.append(target['ID'])
            uncertain_nymg.append(target['YNMG'])
            uncertain_ra.append(target['RA'])
            uncertain_dec.append(target['DEC'])
            uncertain_mag.append(target['phot_g_mean_mag'])
            uncertain_rv.append(target['radial_velocity'])
            uncertain_teff.append(target['teff_val'])
            uncertain_rad.append(target['radius_val'])
            uncertain_lum.append(target['lum_val'])
            uncertain_dist.append(target['distances'])
            uncertain_obs.append(target['fraction_obs'])
            uncertain_exp.append(target['exposure_time_sec'])
     
    uncertain_table = Table([uncertain_id, uncertain_nymg, uncertain_ra, uncertain_dec, uncertain_mag, uncertain_rv, uncertain_teff, uncertain_rad, 
                                   uncertain_lum, uncertain_dist, uncertain_obs, uncertain_exp], 
                                    names = ('ID', 'YNMG', 'RA', 'DEC', 'phot_mean_g_mag', 'radial_velocity', 'teff_val', 'radius_val', 'lum_val', 'distances', 
                                               'fraction_obs', 'exposure_time_sec'))
    uncertain_table['distances'] = np.round(uncertain_table['distances'], 2)
    uncertain_table['fraction_obs'] = np.round(uncertain_table['fraction_obs'], 2)
    
    return uncertain_table

if location.name == 'spm':
    bpmg_from_spm = bpmg_from_obs()
    ascii.write(bpmg_from_spm, format='latex')  
    bpmg_from_spm.write('./tables/bpmg_from_spm_no_vr.csv', overwrite=True)
    
    uncertain_from_spm = uncertain_from_obs()
    ascii.write(uncertain_from_spm, format='latex')  
    uncertain_from_spm.write('./tables/uncertain_from_spm_no_vr.csv', overwrite=True)
    
if location.name == 'CASLEO':
    bpmg_from_casleo = bpmg_from_obs()
    ascii.write(bpmg_from_casleo, format='latex')  
    bpmg_from_casleo.write('./tables/bpmg_from_casleo_no_vr.csv', overwrite=True)
    
    uncertain_from_casleo = uncertain_from_obs()
    ascii.write(uncertain_from_casleo, format='latex')  
    uncertain_from_casleo.write('./tables/uncertain_from_casleo_no_vr.csv', overwrite=True)
    
if location.name == 'Kitt Peak':    
    bpmg_from_kpno = bpmg_from_obs()
    ascii.write(bpmg_from_kpno, format='latex')  
    bpmg_from_kpno.write('./tables/bpmg_from_kpno_no_vr.csv', overwrite=True)
    
    uncertain_from_kpno = uncertain_from_obs()
    ascii.write(uncertain_from_kpno, format='latex')  
    uncertain_from_kpno.write('./tables/uncertain_from_kpno_no_vr.csv', overwrite=True)
    
if location.name == 'Cerro Pachon':
    bpmg_from_soar = bpmg_from_obs()
    ascii.write(bpmg_from_soar, format='latex')  
    bpmg_from_soar.write('./tables/bpmg_from_soar_no_vr.csv', overwrite=True)
    
    uncertain_from_soar = uncertain_from_obs()
    ascii.write(uncertain_from_soar, format='latex')  
    uncertain_from_soar.write('./tables/uncertain_from_soar_no_vr.csv', overwrite=True)